In [1]:
# prepare data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, json
import glob
import operator
from IPython.display import display, HTML

In [2]:
%load_ext autoreload
%autoreload 1

from shared_globals_functions import *
%aimport shared_globals_functions

In [3]:
t_name='t1'
merge_list=['t1.1', 't1.2', 't1.3', 't1.4']

cust_dir = 'test'

tdidf_max_df=1.0 # defaults to 1.0 (float), can be an int too
tdidf_min_df=4
use_tfidfvectorizer = True
use_stopwords = True
use_extended_stopwords = False

umap_metric='hellinger'
umap_ncomponents=2
umap_random_state=42
umap_min_dist=0 #0.1
umap_n_neighbors=30 #10, 30

min_cluster=10
max_cluster=30
clust_int=3

ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50

#tf-idf vectorizer
#max_df=0.3
#min_df=

#kmeans
#n_clusters
#n_init
#max_iter

In [4]:
# Parameters
cust_dir = "ar2/"
use_tfidfvectorizer = False
tdidf_max_df = 1.0
tdidf_min_df = 4
use_stopwords = True
use_extended_stopwords = False
umap_metric = "hellinger"
umap_ncomponents = 2
umap_random_state = 42
umap_min_dist = 0
umap_n_neighbors = 30
min_cluster = 10
max_cluster = 30
clust_int = 3
ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50
t_name = "t2.5.1s"
merge_list = ""


In [5]:
testset = from_disk(t_name)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.5.1s.pkl


In [6]:
len(testset[testset['topics'].str.len() > 0])

218378

In [7]:
if len(merge_list)>0:
    t_name = f'{t_name}+{"+".join([re.sub(r"t", "", tid) for tid in merge_list])}'
    print(t_name)

In [8]:
testset_merge = create_merge(testset, merge_list, merge_to_one=True)

In [9]:
len(testset_merge[testset_merge['topics'].str.len() > 0])

218378

In [10]:
df = testset_merge.merge(testset, on=['name'], how='left', indicator=True)

In [11]:
df[df['_merge']!='both']

,name,topics_x,topics_y,_merge


In [12]:
df[df['topics_x'].str.len() == 0]

,name,topics_x,topics_y,_merge


In [13]:
if len(merge_list)>0:
    testset = testset_merge

In [14]:
import pandas as pd
import umap
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [15]:
orig_testset = testset

In [16]:
my_stop_words=None
if use_stopwords:
    my_stop_words='english'
    if use_extended_stopwords:
        print("todo")

if use_tfidfvectorizer:
    vectorizer = TfidfVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)
else:
    vectorizer = CountVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)

print(f"total repos {len(testset[testset['topics'].str.len() > 0])}")
testset = testset[testset['topics'].str.len() > 0].reset_index()

if isinstance(testset['topics'][0], list):
    testset['topics'] = testset['topics'].apply(lambda x: ' '.join(x))
    print("topics came as list ... consider joining it before")

cr_start()
word_doc_matrix = vectorizer.fit_transform(testset['topics'])
runtime_vectorizer = cr_getrt()

total repos 218378


topics came as list ... consider joining it before


In [17]:
runtime_vectorizer

19.665651082992554

In [18]:
word_doc_matrix

<218378x9454 sparse matrix of type '<class 'numpy.int64'>'
	with 35503466 stored elements in Compressed Sparse Row format>

In [19]:
class ProgressWriter:
    def write(self, text):
        print("custom progress", n, total)

    def flush(self):
        pass

#tqdm_kwds = {"file": "progress_writer", "disable": False }
tqdm_kwds = {"disable": False }   


In [20]:
%%time
# try also with default parameters for n_neighbors and min_dist

cr_start()

embedding = umap.UMAP(
    n_neighbors=umap_n_neighbors,
    min_dist=umap_min_dist,
    n_components=umap_ncomponents, metric=umap_metric,
    tqdm_kwds=tqdm_kwds,
    random_state=umap_random_state,
).fit(word_doc_matrix)

runtime_umap = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/umap/umap_.py:125: UserWarning: A few of your vertices were disconnected from the manifold.  This shouldn't cause problems.
Disconnection_distance = 1 has removed 507 edges.
It has only fully disconnected 7 vertices.
Use umap.utils.disconnected_vertices() to identify them.
  warn(


Epochs completed:   0%|            0/200 [00:00]

CPU times: user 1h 52min 56s, sys: 8min 25s, total: 2h 1min 22s
Wall time: 24min 49s


In [21]:
print(runtime_umap)

1489.1650886535645


In [22]:
%%time

cr_start()
clusterable_embedding = embedding.transform(word_doc_matrix)
runtime_umap_transform = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Epochs completed:   0%|            0/30 [00:00]

CPU times: user 4h 33min 11s, sys: 59.7 s, total: 4h 34min 10s
Wall time: 4h 29min 28s


In [23]:
print(runtime_umap_transform)

16168.35038137436


In [24]:
umap_embedding = pd.DataFrame(embedding.embedding_, columns=['x', 'y']) # (umap_t)
umap_clusterable_embedding = pd.DataFrame(clusterable_embedding, columns=['x', 'y']) # (umap_t)

umap_embedding['name'] = testset['name']
umap_clusterable_embedding['name'] = testset['name']

to_disk_results(umap_embedding, f'umap_embedding_{t_name}', subdir=f"{cust_dir}")
to_disk_results(umap_clusterable_embedding, f'umap_clusterable_embedding_{t_name}', subdir=f"{cust_dir}")

writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//umap_embedding_t2.5.1s.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//umap_clusterable_embedding_t2.5.1s.pkl


https://hdbscan.readthedocs.io/en/latest/parameter_selection.html

Since we have seen that min_samples clearly has a dramatic effect on clustering, the question becomes: how do we select this parameter? The simplest intuition for what min_samples does is provide a measure of how conservative you want you clustering to be. The larger the value of min_samples you provide, the more conservative the clustering – more points will be declared as noise, and clusters will be restricted to progressively more dense areas. We can see this in practice by leaving the min_cluster_size at 60, but reducing min_samples to 1.

In [25]:
import hdbscan

def clusterapply_hdbscan(hdbscan_min_samples, min_cluster_size, runner_name):
    cr_start()
    labels = hdbscan.HDBSCAN(
        min_samples=hdbscan_min_samples,
        min_cluster_size=min_cluster_size,
    ).fit_predict(clusterable_embedding)
    runtime_hdb = cr_getrt()
    
    stats = {'type': "hdb",
             'testset': t_name,
             'version': td_version,
            }
    
    detail_stats = {'hdb_minclust': min_cluster_size,
                    'hdb_minsamp': hdbscan_min_samples,
                    'runtime': runtime_hdb,
                    'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                    'umap': f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}",
                    #'gtr': grtr,
                    #'gtr_rel': grtr_rt
                   }
    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'tot_anom':no_anomalies, 'no_lab':missing_clabel, 'anom':identified_anom})
    prediction.update(detail_stats)
    res.append(prediction.copy())
    display(pd.DataFrame.from_dict(res))
    return labels

res = []
hdb_labels_results = pd.DataFrame()
hdb_labels_results['name'] = testset['name'].copy()


for cs in range(cs_min, cs_max, cs_int): #50, 500
    for ss in range(ss_min, ss_max, ss_int): #50 500
        runner_name = f'ss{ss}cs{cs}'
        hdb_labels_results[runner_name] = clusterapply_hdbscan(ss, cs, runner_name)

results = pd.DataFrame.from_dict(res)
res = []


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s,1,ss50cs50,50,50,5.225229,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s,1,ss50cs50,50,50,5.225229,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
1,hdb,t2.5.1s,1,ss100cs50,50,100,5.138580,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s,1,ss50cs50,50,50,5.225229,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
1,hdb,t2.5.1s,1,ss100cs50,50,100,5.138580,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
2,hdb,t2.5.1s,1,ss150cs50,50,150,5.590156,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s,1,ss50cs50,50,50,5.225229,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
1,hdb,t2.5.1s,1,ss100cs50,50,100,5.138580,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
2,hdb,t2.5.1s,1,ss150cs50,50,150,5.590156,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
3,hdb,t2.5.1s,1,ss200cs50,50,200,6.406808,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s,1,ss50cs50,50,50,5.225229,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
1,hdb,t2.5.1s,1,ss100cs50,50,100,5.138580,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
2,hdb,t2.5.1s,1,ss150cs50,50,150,5.590156,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
3,hdb,t2.5.1s,1,ss200cs50,50,200,6.406808,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
4,hdb,t2.5.1s,1,ss250cs50,50,250,6.944702,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."


...

,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s,1,ss50cs50,50,50,5.225229,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
1,hdb,t2.5.1s,1,ss100cs50,50,100,5.138580,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
2,hdb,t2.5.1s,1,ss150cs50,50,150,5.590156,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
3,hdb,t2.5.1s,1,ss200cs50,50,200,6.406808,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
4,hdb,t2.5.1s,1,ss250cs50,50,250,6.944702,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
...,...,...,...,...,...,...,...,...,...
72,hdb,t2.5.1s,1,ss50cs450,450,50,4.469547,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
73,hdb,t2.5.1s,1,ss100cs450,450,100,5.175207,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
74,hdb,t2.5.1s,1,ss150cs450,450,150,5.474105,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
75,hdb,t2.5.1s,1,ss200cs450,450,200,6.063262,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s,1,ss50cs50,50,50,5.225229,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
1,hdb,t2.5.1s,1,ss100cs50,50,100,5.138580,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
2,hdb,t2.5.1s,1,ss150cs50,50,150,5.590156,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
3,hdb,t2.5.1s,1,ss200cs50,50,200,6.406808,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
4,hdb,t2.5.1s,1,ss250cs50,50,250,6.944702,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
...,...,...,...,...,...,...,...,...,...
73,hdb,t2.5.1s,1,ss100cs450,450,100,5.175207,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
74,hdb,t2.5.1s,1,ss150cs450,450,150,5.474105,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
75,hdb,t2.5.1s,1,ss200cs450,450,200,6.063262,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
76,hdb,t2.5.1s,1,ss250cs450,450,250,6.947330,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s,1,ss50cs50,50,50,5.225229,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
1,hdb,t2.5.1s,1,ss100cs50,50,100,5.138580,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
2,hdb,t2.5.1s,1,ss150cs50,50,150,5.590156,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
3,hdb,t2.5.1s,1,ss200cs50,50,200,6.406808,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
4,hdb,t2.5.1s,1,ss250cs50,50,250,6.944702,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
...,...,...,...,...,...,...,...,...,...
74,hdb,t2.5.1s,1,ss150cs450,450,150,5.474105,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
75,hdb,t2.5.1s,1,ss200cs450,450,200,6.063262,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
76,hdb,t2.5.1s,1,ss250cs450,450,250,6.947330,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
77,hdb,t2.5.1s,1,ss300cs450,450,300,7.670837,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s,1,ss50cs50,50,50,5.225229,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
1,hdb,t2.5.1s,1,ss100cs50,50,100,5.138580,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
2,hdb,t2.5.1s,1,ss150cs50,50,150,5.590156,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
3,hdb,t2.5.1s,1,ss200cs50,50,200,6.406808,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
4,hdb,t2.5.1s,1,ss250cs50,50,250,6.944702,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
...,...,...,...,...,...,...,...,...,...
75,hdb,t2.5.1s,1,ss200cs450,450,200,6.063262,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
76,hdb,t2.5.1s,1,ss250cs450,450,250,6.947330,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
77,hdb,t2.5.1s,1,ss300cs450,450,300,7.670837,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
78,hdb,t2.5.1s,1,ss350cs450,450,350,7.845774,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s,1,ss50cs50,50,50,5.225229,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
1,hdb,t2.5.1s,1,ss100cs50,50,100,5.138580,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
2,hdb,t2.5.1s,1,ss150cs50,50,150,5.590156,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
3,hdb,t2.5.1s,1,ss200cs50,50,200,6.406808,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
4,hdb,t2.5.1s,1,ss250cs50,50,250,6.944702,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
...,...,...,...,...,...,...,...,...,...
76,hdb,t2.5.1s,1,ss250cs450,450,250,6.947330,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
77,hdb,t2.5.1s,1,ss300cs450,450,300,7.670837,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
78,hdb,t2.5.1s,1,ss350cs450,450,350,7.845774,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."
79,hdb,t2.5.1s,1,ss400cs450,450,400,8.556778,"rt_vect: 19.665651082992554, min_df: 4, max_df...","hellinger, rt_umap_tf: 16168.35038137436, rt_u..."


In [26]:
hdb_labels_results

,name,ss50cs50,ss100cs50,ss150cs50,ss200cs50,ss250cs50,ss300cs50,ss350cs50,ss400cs50,ss450cs50,...,ss450cs400,ss50cs450,ss100cs450,ss150cs450,ss200cs450,ss250cs450,ss300cs450,ss350cs450,ss400cs450,ss450cs450
0,technopagan/mss-saliency,56,31,19,11,7,6,5,4,2,...,1,3,2,3,2,2,1,1,1,1
1,snmnmin12/selective-search,202,29,15,51,7,6,5,4,2,...,1,3,4,4,2,2,1,1,1,1
2,google/chaps-linux,-1,-1,-1,-1,7,6,5,4,2,...,1,3,2,3,2,2,1,1,1,1
3,lukasolson/speech-router,-1,-1,-1,-1,7,6,5,4,2,...,1,3,2,3,2,2,1,1,1,1
4,XavierTecBr/qbr-admin,179,95,-1,-1,7,6,5,4,2,...,1,3,2,3,2,2,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218373,bhagatabhijeet/bhagatabhijeet,-1,-1,-1,-1,7,6,5,4,2,...,1,3,2,3,2,2,1,1,1,1
218374,LandGrey/SpringBootVulExploit,28,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,3,-1,-1,-1,-1,-1,-1,-1,-1
218375,mucomplex/Pentest_checklist,184,93,61,50,7,6,5,4,2,...,1,3,2,3,2,2,1,1,1,1
218376,bigsen/Vision_Track,-1,-1,-1,-1,7,6,5,4,2,...,1,3,2,3,2,2,1,1,1,1


In [27]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/hdb_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'hdb_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'hdb_results', subdir=f"{cust_dir}")
to_disk_results(hdb_labels_results, f'hdb_labels_results_{t_name}', subdir=f"{cust_dir}")

saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//hdb_labels_results_t2.5.1s.pkl


In [28]:

umap_embedding['topics'] = testset['topics']
umap_clusterable_embedding['topics'] = testset['topics']

In [29]:
import sklearn.cluster as cluster

def clusterapply_kmeans(testset, clustersize, runner_name, opt_comment=""):
    # Perform k-means clustering
    
    kmeans = cluster.KMeans(n_clusters=clustersize, random_state=42)
        
    if('y' in testset.columns):
        #print("assuming umap reduced testset here")
        testset = testset.copy()
        dropping_not_connected = len(testset[testset['y'].isnull()])
        testset = testset[testset['y'].notna()]
        cr_start()
        kmeans.fit(testset[['x', 'y']].to_numpy())
        runtime_kmeans = cr_getrt()
        opt_comment="-umapt"
    else:
        dropping_not_connected = 0
        cr_start()
        kmeans.fit(word_doc_matrix)
        runtime_kmeans = cr_getrt()

    
    stats = {'type': f"kmeans{opt_comment}",
             'testset': t_name,
             'version': td_version,
             'size': clustersize
            }
    
    detail_stats = {'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                'runtime': runtime_kmeans
               }
    
    if('y' in testset.columns):
        detail_stats['umap'] = f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}", #umap not important here
        stats['no_con'] = dropping_not_connected

    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'no_lab':missing_clabel})
    prediction.update(detail_stats)
    res.append(prediction.copy())
        
    display(pd.DataFrame.from_dict(res))
    return kmeans.labels_
    #testset.drop(['cluster_run'], axis=1)


In [30]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = umap_embedding[umap_embedding['y'].notna()]['name'].copy()

for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(umap_embedding, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix



,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s,1,10,7,10,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.257779,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s,1,10,7,10,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.257779,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
1,kmeans-umapt,t2.5.1s,1,13,7,13,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.403641,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s,1,10,7,10,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.257779,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
1,kmeans-umapt,t2.5.1s,1,13,7,13,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.403641,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
2,kmeans-umapt,t2.5.1s,1,16,7,16,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.765576,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s,1,10,7,10,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.257779,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
1,kmeans-umapt,t2.5.1s,1,13,7,13,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.403641,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
2,kmeans-umapt,t2.5.1s,1,16,7,16,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.765576,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
3,kmeans-umapt,t2.5.1s,1,19,7,19,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.930806,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s,1,10,7,10,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.257779,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
1,kmeans-umapt,t2.5.1s,1,13,7,13,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.403641,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
2,kmeans-umapt,t2.5.1s,1,16,7,16,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.765576,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
3,kmeans-umapt,t2.5.1s,1,19,7,19,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.930806,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
4,kmeans-umapt,t2.5.1s,1,22,7,22,"rt_vect: 19.665651082992554, min_df: 4, max_df...",2.433762,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s,1,10,7,10,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.257779,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
1,kmeans-umapt,t2.5.1s,1,13,7,13,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.403641,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
2,kmeans-umapt,t2.5.1s,1,16,7,16,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.765576,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
3,kmeans-umapt,t2.5.1s,1,19,7,19,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.930806,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
4,kmeans-umapt,t2.5.1s,1,22,7,22,"rt_vect: 19.665651082992554, min_df: 4, max_df...",2.433762,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
5,kmeans-umapt,t2.5.1s,1,25,7,25,"rt_vect: 19.665651082992554, min_df: 4, max_df...",2.634947,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s,1,10,7,10,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.257779,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
1,kmeans-umapt,t2.5.1s,1,13,7,13,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.403641,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
2,kmeans-umapt,t2.5.1s,1,16,7,16,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.765576,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
3,kmeans-umapt,t2.5.1s,1,19,7,19,"rt_vect: 19.665651082992554, min_df: 4, max_df...",1.930806,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
4,kmeans-umapt,t2.5.1s,1,22,7,22,"rt_vect: 19.665651082992554, min_df: 4, max_df...",2.433762,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
5,kmeans-umapt,t2.5.1s,1,25,7,25,"rt_vect: 19.665651082992554, min_df: 4, max_df...",2.634947,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."
6,kmeans-umapt,t2.5.1s,1,28,7,28,"rt_vect: 19.665651082992554, min_df: 4, max_df...",3.071838,"(hellinger, rt_umap_tf: 16168.35038137436, rt_..."


In [31]:
kmeans_labels_results

,name,10,13,16,19,22,25,28
0,technopagan/mss-saliency,0,9,4,5,19,24,5
1,snmnmin12/selective-search,1,1,15,17,10,15,12
2,google/chaps-linux,9,7,10,13,13,1,27
3,lukasolson/speech-router,5,2,12,8,14,16,14
4,XavierTecBr/qbr-admin,9,7,2,3,15,2,24
...,...,...,...,...,...,...,...,...
218373,bhagatabhijeet/bhagatabhijeet,2,3,5,6,2,18,7
218374,LandGrey/SpringBootVulExploit,6,11,11,14,16,0,4
218375,mucomplex/Pentest_checklist,9,7,10,18,13,9,27
218376,bigsen/Vision_Track,0,9,4,5,19,24,5


In [32]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/kmeans-umap_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans-umap_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans-umap_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans-umap_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//kmeans-umap_labels_results_t2.5.1s.pkl


In [33]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = testset['name'].copy()


for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(testset, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix

,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s,1,10,10,"rt_vect: 19.665651082992554, min_df: 4, max_df...",168.610951


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s,1,10,10,"rt_vect: 19.665651082992554, min_df: 4, max_df...",168.610951
1,kmeans,t2.5.1s,1,13,13,"rt_vect: 19.665651082992554, min_df: 4, max_df...",202.453041


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s,1,10,10,"rt_vect: 19.665651082992554, min_df: 4, max_df...",168.610951
1,kmeans,t2.5.1s,1,13,13,"rt_vect: 19.665651082992554, min_df: 4, max_df...",202.453041
2,kmeans,t2.5.1s,1,16,16,"rt_vect: 19.665651082992554, min_df: 4, max_df...",294.404695


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s,1,10,10,"rt_vect: 19.665651082992554, min_df: 4, max_df...",168.610951
1,kmeans,t2.5.1s,1,13,13,"rt_vect: 19.665651082992554, min_df: 4, max_df...",202.453041
2,kmeans,t2.5.1s,1,16,16,"rt_vect: 19.665651082992554, min_df: 4, max_df...",294.404695
3,kmeans,t2.5.1s,1,19,19,"rt_vect: 19.665651082992554, min_df: 4, max_df...",438.263465


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s,1,10,10,"rt_vect: 19.665651082992554, min_df: 4, max_df...",168.610951
1,kmeans,t2.5.1s,1,13,13,"rt_vect: 19.665651082992554, min_df: 4, max_df...",202.453041
2,kmeans,t2.5.1s,1,16,16,"rt_vect: 19.665651082992554, min_df: 4, max_df...",294.404695
3,kmeans,t2.5.1s,1,19,19,"rt_vect: 19.665651082992554, min_df: 4, max_df...",438.263465
4,kmeans,t2.5.1s,1,22,22,"rt_vect: 19.665651082992554, min_df: 4, max_df...",421.216398


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s,1,10,10,"rt_vect: 19.665651082992554, min_df: 4, max_df...",168.610951
1,kmeans,t2.5.1s,1,13,13,"rt_vect: 19.665651082992554, min_df: 4, max_df...",202.453041
2,kmeans,t2.5.1s,1,16,16,"rt_vect: 19.665651082992554, min_df: 4, max_df...",294.404695
3,kmeans,t2.5.1s,1,19,19,"rt_vect: 19.665651082992554, min_df: 4, max_df...",438.263465
4,kmeans,t2.5.1s,1,22,22,"rt_vect: 19.665651082992554, min_df: 4, max_df...",421.216398
5,kmeans,t2.5.1s,1,25,25,"rt_vect: 19.665651082992554, min_df: 4, max_df...",603.986188


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s,1,10,10,"rt_vect: 19.665651082992554, min_df: 4, max_df...",168.610951
1,kmeans,t2.5.1s,1,13,13,"rt_vect: 19.665651082992554, min_df: 4, max_df...",202.453041
2,kmeans,t2.5.1s,1,16,16,"rt_vect: 19.665651082992554, min_df: 4, max_df...",294.404695
3,kmeans,t2.5.1s,1,19,19,"rt_vect: 19.665651082992554, min_df: 4, max_df...",438.263465
4,kmeans,t2.5.1s,1,22,22,"rt_vect: 19.665651082992554, min_df: 4, max_df...",421.216398
5,kmeans,t2.5.1s,1,25,25,"rt_vect: 19.665651082992554, min_df: 4, max_df...",603.986188
6,kmeans,t2.5.1s,1,28,28,"rt_vect: 19.665651082992554, min_df: 4, max_df...",716.296851


In [34]:
kmeans_labels_results

,name,10,13,16,19,22,25,28
0,technopagan/mss-saliency,7,8,4,6,1,0,0
1,snmnmin12/selective-search,3,9,10,8,9,11,11
2,google/chaps-linux,4,6,15,9,11,3,3
3,lukasolson/speech-router,3,9,10,8,1,0,0
4,XavierTecBr/qbr-admin,3,9,10,8,9,11,11
...,...,...,...,...,...,...,...,...
218373,bhagatabhijeet/bhagatabhijeet,3,9,10,8,9,11,11
218374,LandGrey/SpringBootVulExploit,7,6,15,9,11,3,19
218375,mucomplex/Pentest_checklist,7,8,4,6,11,0,0
218376,bigsen/Vision_Track,3,9,10,8,9,11,11


In [35]:
print("saving results to disk")

try:
    old_results = from_disk(f'results/{cust_dir}/kmeans_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar2//kmeans_labels_results_t2.5.1s.pkl
